## Preparing A Dataset for Modeling  
### 04-Apr-2019 Lou Pendley  
### Version 1.0

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [21]:
# Let's read in the data set
DataFilePath = '../../../../Datafiles'
df = pd.read_csv(DataFilePath + '/imdb-data-from-2006-to-2016.zip', 
                 compression='zip', header=0, sep=',', quotechar='"')
df['first_genre'] = df['Genre'].apply(lambda x: x.split(',')[0])
df['first_actor'] = df['Actors'].apply(lambda x: x.split(',')[0])

In [38]:
df['first_actor'].head(2)

507    Zooey Deschanel
30         Zoey Deutch
Name: first_actor, dtype: object

In [36]:
df['first_genre'].sample(5)

535     Action
733    Mystery
430     Comedy
580     Action
524      Crime
Name: first_genre, dtype: object

In [54]:
# print(df['first_actor'].unique())
sort_by = ['first_actor']

# We will want to group by Station ID: start, and end
df_sorted_actor=pd.DataFrame()
df_sorted_actor=df.sort_values(by=sort_by, ascending=False)
# print(df_sorted_actor)
df_sorted_actor.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore', 'first_genre', 'first_actor'],
      dtype='object')

In [82]:
df_sorted_genre = df.sort_values(by=['first_genre'])
print("there are {} distinct genres.".format(len(df_sorted_genre['first_genre'].unique())))
for name in df_sorted_genre['first_genre'].unique():
    print("first genre is {}".format(name))

there are 13 distinct genres.
first genre is Action
first genre is Adventure
first genre is Animation
first genre is Biography
first genre is Comedy
first genre is Crime
first genre is Drama
first genre is Fantasy
first genre is Horror
first genre is Mystery
first genre is Romance
first genre is Sci-Fi
first genre is Thriller


In [86]:
i = 1
df_sorted_first_actor = df.sort_values(by=['first_actor'])
print("there are {} distinct first actors.".format(len(df_sorted_first_actor['first_actor'].unique())))
for name in df_sorted_first_actor['first_actor'].unique():
    print("first actor number {} is {}".format(i, name))
    i += 1

there are 525 distinct first actors.
first actor number 1 is Aamir Khan
first actor number 2 is Aaron Paul
first actor number 3 is Aaron Poole
first actor number 4 is Aaron Taylor-Johnson
first actor number 5 is Abbie Cornish
first actor number 6 is Abigail Breslin
first actor number 7 is Adam Driver
first actor number 8 is Adam Pally
first actor number 9 is Adam Sandler
first actor number 10 is Adrian Grenier
first actor number 11 is Adrian Titieni
first actor number 12 is Adrien Brody
first actor number 13 is Alex Karpovsky
first actor number 14 is Alex Pettyfer
first actor number 15 is Alexander Skarsgård
first actor number 16 is Alexandra Daddario
first actor number 17 is Alexi Pappas
first actor number 18 is Alia Bhatt
first actor number 19 is Alia Shawkat
first actor number 20 is Alice Englert
first actor number 21 is Alicia Vikander
first actor number 22 is Alison Brie
first actor number 23 is Alycia Debnam-Carey
first actor number 24 is Amanda Bynes
first actor number 25 is Amb

In [65]:
grouping_columns = ['first_actor']
sort_by_aggregate = ['count']

# # We will want to group by first actor
df_first_actor_grouped=df.groupby(grouping_columns )['first_actor'].agg(['count']) 
df_first_actor_grouped=df_first_actor_grouped.sort_values(by=sort_by_aggregate, ascending=False)
print(df_first_actor_grouped)

In [67]:
grouping_columns = ['first_genre']
sort_by_aggregate = ['count']

# # We will want to group by first genre
df_first_genre_grouped=df.groupby(grouping_columns )['first_actor'].agg(['count']) 
df_first_genre_grouped=df_first_genre_grouped.sort_values(by=sort_by_aggregate, ascending=False)
print(df_first_genre_grouped)

             count
first_genre       
Action         293
Drama          195
Comedy         175
Adventure       75
Crime           71
Biography       64
Animation       49
Horror          46
Mystery         13
Thriller        10
Fantasy          4
Sci-Fi           3
Romance          2
